Created on Fri Jan 10 11:06:30 2020
<br>
Group 7
<br>
@author: E.G.

<h1>Group 7 - Images sociales<span class="tocSkip"></span>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Functions</a></span></li></ul></li><li><span><a href="#Pre-processing" data-toc-modified-id="Pre-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pre-processing</a></span><ul class="toc-item"><li><span><a href="#Train-test-split-and-read-images" data-toc-modified-id="Train-test-split-and-read-images-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Train-test split and read images</a></span></li></ul></li><li><span><a href="#Build,-save,-and-train-model" data-toc-modified-id="Build,-save,-and-train-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build, save, and train model</a></span></li></ul></div>

# Introduction
This notebook trains a model to predict Airbus aircraft types on images representing an aircraft interior. The images used for training come from Hackathon data ("clean" images of Airbus aircraft interiors).

**Pre-processing**<span class="tocSkip"></span><br>
By reading and filtering the CSV file that contains the labels (Int, Ext, Ext-Int, Meal), we get the list of Interior labelled images. Then, the images are copied to directories (one per desired aircraft type), and split into train and test sets. If the data augmentation option is set to `True`, the train set will be enriched with new images (obtained by cropping / (de)zooming / rotating / flipping existing images).

**Model**<span class="tocSkip"></span><br>
We get weights from VGG16 pre-trained model, and add some layers (Conv2D, ReLU, MaxPooling2D, Flatten, and Dense) to predict the target classes (e.g.: 3 classes could be A320, A350, and A380).

**Out**<span class="tocSkip"></span><br>
After training, a folder is created in `Models` repository, containing the model in `h5` format, along with the corresponding labels stored in a `pickle` dictionary.

# Environment
To ensure a proper functioning of this code file, `python 3.6` or later version is required.
## Libraries

In [2]:
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.models import Sequential, Model
from keras import backend as K
import keras
import pandas as pd
import random
import os
from PIL import Image
from shutil import copyfile
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:
%load_ext watermark
%watermark -p keras,tensorflow,PIL

keras 2.3.1
tensorflow 1.13.1
PIL 6.2.0


## Parameters

In [20]:
project_path = './../'
seatguru_path = './../All Data/ANALYSE IMAGE/IMG SEATGURU/'
hackathon_path = './../Inputs Hackathon/'
new_path_train = './../Split_data/Hackathon/Int/data_train'
new_path_val = './../Split_data/Hackathon/Int/data_val'
new_paths = [new_path_train, new_path_val]
model_name = 'Int_Airbus_Hackathon'

# Images parameters
size = (224, 224)
greys = False
apply_data_augmentation = True

# Classes to predict
airbus_planes = ['A320', 'A350', 'A380']
nb_types = len(airbus_planes)

## Functions

In [19]:
%run g7_functions_for_models.ipynb

# Pre-processing
## Train-test split and read images

In [21]:
split_train_test_hack(new_paths, hackathon_path,
                      airbus_planes=airbus_planes, split_limit=.7, s=8)

In [12]:
# Image data generator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(new_paths[0],
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True)

test_generator = train_datagen.flow_from_directory(new_paths[1],
                                                   target_size=(224, 224),
                                                   color_mode='rgb',
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   shuffle=True)

Found 618 images belonging to 3 classes.
Found 267 images belonging to 3 classes.


# Build, save, and train model

In [14]:
# Get the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False,
                   input_shape=(size[0], size[1], 1 if greys else 3))
x = base_model.output

# Add layers
x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)  # vector

x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Last layer used to predict our classes
# Dense has the same number of neurons as the number of classes to predict
predictions = Dense(nb_types, activation='softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done after setting layers to non-trainable)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [67]:
# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',  # chosen metric
                              patience=2,  # number of epochs
                              verbose=1,
                              factor=0.5,
                              min_lr=0.00001)

In [68]:
# Train model
step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])

Epoch 1/10
25/25 [==============================] - ETA: 16:00 - loss: 0.2860 - accuracy: 0.875 - ETA: 14:17 - loss: 0.2802 - accuracy: 0.906 - ETA: 13:26 - loss: 0.2479 - accuracy: 0.916 - ETA: 12:02 - loss: 0.2400 - accuracy: 0.914 - ETA: 11:01 - loss: 0.2803 - accuracy: 0.900 - ETA: 10:05 - loss: 0.2605 - accuracy: 0.911 - ETA: 9:13 - loss: 0.2924 - accuracy: 0.901 - ETA: 8:34 - loss: 0.2965 - accuracy: 0.90 - ETA: 7:55 - loss: 0.2885 - accuracy: 0.90 - ETA: 7:18 - loss: 0.2650 - accuracy: 0.91 - ETA: 6:44 - loss: 0.2436 - accuracy: 0.92 - ETA: 6:12 - loss: 0.2380 - accuracy: 0.91 - ETA: 5:41 - loss: 0.2270 - accuracy: 0.92 - ETA: 5:11 - loss: 0.2269 - accuracy: 0.92 - ETA: 4:42 - loss: 0.2311 - accuracy: 0.92 - ETA: 4:12 - loss: 0.2284 - accuracy: 0.91 - ETA: 3:42 - loss: 0.2575 - accuracy: 0.90 - ETA: 3:13 - loss: 0.2652 - accuracy: 0.90 - ETA: 2:44 - loss: 0.2748 - accuracy: 0.90 - ETA: 2:15 - loss: 0.2760 - accuracy: 0.90 - ETA: 1:47 - loss: 0.2877 - accuracy: 0.90 - ETA: 1:20 -

In [ ]:
# Save model and labels
os.makedirs(project_path + 'Models/' + model_name + '/', exist_ok=True)
save_model_classes(project_path + 'Models/',
                   model_name, train_generator, model)